#Установка CityGeoTools

In [1]:
!git clone https://github.com/iduprojects/CityGeoTools

Cloning into 'CityGeoTools'...
remote: Enumerating objects: 2775, done.
remote: Counting objects: 100% (1030/1030), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 2775 (delta 690), reused 850 (delta 636), pack-reused 1745
Receiving objects: 100% (2775/2775), 114.30 MiB | 15.76 MiB/s, done.
Resolving deltas: 100% (1915/1915), done.


In [2]:
!pip install -r /content/CityGeoTools/metrics/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 668.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.0/243.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install scipy
!pip install rpyc
!pip install networkit
!pip install pca

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 17.0 MB/s eta 0:00:00
  Using cached pca-2.0.5-py3-none-any.whl (35 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.3 MB/s eta 0:00:00


In [4]:
import sys
import os
import numpy as np
import geopandas as gpd
import folium
import json
import pandas as pd
from CityGeoTools.metrics.data import CityInformationModel as BaseModel
from CityGeoTools.metrics.calculations import services_clusterization

#Код

In [30]:
bank = gpd.read_file("/content/bank.geojson")
cinema = gpd.read_file("/content/cinema.geojson")
fitness_centre = gpd.read_file("/content/fitness_centre.geojson")
hotel = gpd.read_file("/content/hotel.geojson")
school = gpd.read_file("/content/school.geojson")
block = gpd.read_file("/content/city_blocks.geojson")
districts = gpd.read_file("/content/districts.geojson")

In [32]:
block = block.to_crs(32641)
districts = districts.to_crs(block.crs)

In [33]:
block["block_id"] = range(len(block))
districts["districts_id"] = range(len(districts))

In [35]:
bank["id"] = range(len(bank))
cinema["id"] = range(len(cinema))
fitness_centre["id"] = range(len(fitness_centre))
hotel["id"] = range(len(hotel))
school["id"] = range(len(school))

In [36]:
bank["service_code"] = 'bank'
cinema["service_code"] = 'cinema'
fitness_centre["service_code"] = 'fitness_centre'
hotel["service_code"] = 'hotel'
school["service_code"] = 'school'

In [42]:
service = gpd.GeoDataFrame(pd.concat([bank, cinema, fitness_centre, hotel, school]))
service = service[["id", "service_code", "geometry"]]
service = service.to_crs(block.crs)

In [43]:
service["geometry"] = service["geometry"].centroid

In [44]:
service = service.sjoin(districts, how="left")
service = service[["id_left", "service_code", "geometry", "districts_id"]]

In [45]:
service = service.sjoin(block, how="left")
service = service[["id_left", "service_code", "geometry", "districts_id", "block_id"]]

In [46]:
service = service.rename(columns={'id_left': 'id', 'districts_id': 'administrative_unit_id'})
service["municipality_id"] = 10
service["block_id"] = service["block_id"].fillna(service["block_id"].median())

In [47]:
service['block_id'] = service['block_id'].astype(int)

In [48]:
service = service.to_crs(epsg=4326)
service.crs = 'urn:ogc:def:crs:OGC::CRS84'


service.to_file("Services.geojson", driver="GeoJSON")

#Кластеризация

In [49]:
city_model = BaseModel.CityInformationModel(city_name="Ekaterinburg", city_crs=32641, cwd="CityGeoTools")

In [50]:
city_model.update_layer("Services", "/content/Services.geojson")

Validation of Services layer...
Services layer loaded successfully!


In [51]:
city_model.methods.services_clusterization.message

{'Services': 'Layer matches specification'}

In [52]:
service_clusterization = services_clusterization.ServicesClusterization(city_model).get_clusters_polygon(
    service_types=['bank', 'cinema', 'fitness_centre', 'school', 'hotel'], n_std=2, condition_value=1000
    )

service_points = gpd.GeoDataFrame.from_features(service_clusterization['services']).set_crs(4326)
service_clusters = gpd.GeoDataFrame.from_features(service_clusterization['polygons']).set_crs(4326)

In [53]:
map_center = [56.833094, 60.612603]
map = folium.Map(location=map_center, zoom_start=13, tiles='CartoDB dark_matter')

service_colors = {
    'bank': 'red',
    'cinema': 'blue',
    'fitness_centre': 'black',
    'school': 'green',
    'hotel': 'pink'
}

for _, row in service_points.iterrows():
    service_type = row['service_code']
    color = service_colors.get(service_type, 'gray')

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Service Type: {service_type}"
    ).add_to(map)

folium.GeoJson(
    data=service_clusters,
    tooltip=folium.features.GeoJsonTooltip(['bank', 'cinema', 'fitness_centre', 'school', 'hotel'])
).add_to(map)

legend_html = f"""
<div style="position: fixed; bottom: 50px; left: 50px; z-index:1000; font-size:14px; background-color:white; padding: 10px; border:1px solid grey; border-radius:5px;">
    <p style="margin:0;"><strong>Legend</strong></p>
    {''.join([f'<p style="margin:0;"><span style="color:{color}; font-size:20px;">&bull;</span> {service_type.capitalize()}</p>' for service_type, color in service_colors.items()])}
</div>
"""

map.get_root().html.add_child(folium.Element(legend_html))

map.save("clusters_satellite.html")
